## Probabilidades 

Referencia:

1 . [Probability and Statistics for Computer Scientists](https://www.crcpress.com/Probability-and-Statistics-for-Computer-Scientists-Second-Edition/Baron/9781439875902) de Michael Baron.